<a href="https://colab.research.google.com/github/Ghost-void/PizzaPie/blob/main/Light_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import requests

def fetch_nasa_power_data(latitude, longitude, start_year, end_year):
    """
    Fetches monthly solar and meteorological data from the NASA POWER API for a specific location and time range.

    Args:
        latitude (float): The latitude of the location.
        longitude (float): The longitude of the location.
        start_year (int): The starting year for the data.
        end_year (int): The ending year for the data.

    Returns:
        dict: A dictionary containing the JSON response from the API, or None in case of an error.
              The dictionary includes the header and the data.
    """
    url = 'https://power.larc.nasa.gov/api/temporal/monthly/point'
    headers = {
        'accept': 'application/json'
    }
    params = {
        'start': start_year,
        'end': end_year,
        'latitude': latitude,
        'longitude': longitude,
        'community': 're',  # Resource Efficiency
        'parameters': 'ALLSKY_SFC_SW_DWN,T2M',  # All Sky Surface Shortwave Downward Radiation, 2 Meter Temperature
        'format': 'json',
        'user': 'LightMap',  #Identifies the user
        'header': 'true' # Include header
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from NASA POWER API: {e}")
        return None

# Ensure the definition of organize_nasa_power_data is ABOVE where it's called
def organize_nasa_power_data(api_data):
    """
    Organizes the data fetched from the NASA POWER API into a list of dictionaries,
    where each dictionary represents data for a specific month and year.

    Args:
        api_data (dict): The dictionary containing the JSON response from the API.

    Returns:
        list: A list of dictionaries, or None if the input data is invalid or missing.
    """
    if not api_data or 'data' not in api_data:
        print("Invalid or missing data in the API response.")
        return None

    organized_list = []
    data_content = api_data['data']
    parameters_info = api_data.get('parameters', {}) # Get parameter descriptions if available

    for year, monthly_data in data_content.items():
        for month, values in monthly_data.items():
            month_data = {
                'year': int(year),
                'month': int(month)
            }
            # Add parameter values to the dictionary
            for param, value in values.items():
                 # Use the parameter description if available, otherwise use the parameter name
                param_key = parameters_info.get(param, {}).get('description', param)
                month_data[param_key] = value
            organized_list.append(month_data)

    # Optionally sort the list by year and month
    organized_list.sort(key=lambda x: (x['year'], x['month']))

    return organized_list
    
  if __name__ == "__main__":
    # ... (your existing code) ...

    data = fetch_nasa_power_data(latitude, longitude, start_year, end_year)

  if data:
      print("Raw data returned by API:")
      print(data) # <-- Add this line to see the raw response structure

      organized_data = organize_nasa_power_data(data)
      # ... (rest of your code) ...
  else:
      print("Failed to retrieve data.")

      if organized_data:
          # Print the organized list
          for entry in organized_data:
              print(entry)
      elif organized_data == []:
          print("Failed to organize data.")
      else:
          print("Failed to retrieve data.")
    


In [2]:
import requests
from urllib.parse import urlencode # For constructing debug URLs
import json # Import for debugging output

def fetch_nasa_power_data(latitude, longitude, start_year, end_year):
    """
    Fetches monthly solar and meteorological data from the NASA POWER API for a specific location and time range.

    Args:
        latitude (float): The latitude of the location.
        longitude (float): The longitude of the location.
        start_year (int): The starting year for the data.
        end_year (int): The ending year for the data.

    Returns:
        dict: A dictionary containing the JSON response from the API, or None in case of an error.
              The dictionary includes the header and the data.
    """
    url = 'https://power.larc.nasa.gov/api/temporal/monthly/point'
    headers = {
        'accept': 'application/json'
    }
    params = {
        'start': start_year,
        'end': end_year,
        'latitude': latitude,
        'longitude': longitude,
        'community': 're',  # Resource Efficiency
        'parameters': 'ALLSKY_SFC_SW_DWN,T2M',  # All Sky Surface Shortwave Downward Radiation, 2 Meter Temperature
        'format': 'json',
        'user': 'LightMap',  # Identifies the user
        'header': 'true'  # Include header
    }

    response_obj = None
    try:
        response_obj = requests.get(url, headers=headers, params=params)
        response_obj.raise_for_status()
        return response_obj.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from NASA POWER API: {e}")
        if response_obj is not None:
            print(f"Request URL: {response_obj.url}")
            print(f"Response Status Code: {response_obj.status_code}")
            print(f"Response Text (first 500 chars): {response_obj.text[:500]}")
        else:
            query_string = urlencode(params)
            intended_url = f"{url}?{query_string}"
            print(f"Intended Request URL: {intended_url}")
        print(f"Request Params: {params}")
        return None

def organize_nasa_power_data(api_data):
    """
    Organizes the data fetched from the NASA POWER API into a list of dictionaries,
    where each dictionary represents data for a specific month and year.

    Args:
        api_data (dict): The dictionary containing the JSON response from the API.

    Returns:
        list: A list of dictionaries, or None if the input data is invalid or missing critical structures,
              or an empty list if no data points could be processed.
    """
    if not api_data:
        print("API data is None. Cannot organize.")
        return None

    # Ensure 'properties' and 'parameter' keys exist at the top level
    if 'properties' not in api_data or 'parameter' not in api_data.get('properties', {}):
        print("Error: Missing 'properties' or 'parameter' key in API response.")
        print("API Response Keys:", api_data.keys())
        print("API Response 'properties' Keys:", api_data.get('properties', {}).keys())
        return None

    parameter_data = api_data['properties']['parameter']
    if not parameter_data:
        print("Warning: 'parameter' data is empty. No monthly data to process.")
        return []

    monthly_records = {}
    param_longnames = {}

    # Safely get parameter_information if available in the header
    if 'header' in api_data and 'parameter_information' in api_data['header']:
        param_info = api_data['header']['parameter_information']
        param_longnames = {code: info.get('longname', code) for code, info in param_info.items()}
    else:
        print("Warning: 'parameter_information' not found in API response header. Using parameter codes as names.")


    # Iterate through each parameter (e.g., ALLSKY_SFC_SW_DWN, T2M)
    for param_code, monthly_values_for_param in parameter_data.items():
        # Get the display name for the parameter, defaulting to the code if longname isn't found
        param_display_name = param_longnames.get(param_code, param_code)

        # Ensure the values for this parameter are a dictionary (containing YYYYMM keys)
        if not isinstance(monthly_values_for_param, dict):
            print(f"Warning: Unexpected data format for parameter '{param_code}'. Expected a dictionary of monthly values. Skipping.")
            continue

        # Iterate through the YYYYMM keys and their corresponding values
        for year_month_str, value in monthly_values_for_param.items():
            # Validate and parse the YYYYMM string
            if len(year_month_str) == 6 and year_month_str.isdigit():
                year_str = year_month_str[:4]
                month_str = year_month_str[4:]
                try:
                    year_int = int(year_str)
                    month_int = int(month_str)

                    # Validate month is within a valid range
                    if not (1 <= month_int <= 12):
                        print(f"Warning: Invalid month value '{month_str}' for '{year_month_str}'. Skipping.")
                        continue

                    # Check for missing data indicator (-999) or None
                    if value is not None and value != -999:
                        record_key = (year_int, month_int)
                        if record_key not in monthly_records:
                            monthly_records[record_key] = {
                                'year': year_int,
                                'month': month_int
                            }
                        monthly_records[record_key][param_display_name] = value
                    # else:
                        # print(f"Debug: Skipping missing/null value for {param_display_name} in {year_month_str}")
                except ValueError:
                    print(f"Warning: Could not convert year ('{year_str}') or month ('{month_str}') to integer for '{year_month_str}'. Skipping.")
            else:
                print(f"Warning: Unexpected year-month string format: '{year_month_str}'. Expected YYYYMM. Skipping.")

    organized_list = list(monthly_records.values())
    # Sort the list by year, then by month for chronological order
    organized_list.sort(key=lambda x: (x['year'], x['month']))

    if not organized_list:
        print("Data organized, but no valid monthly entries were found after processing.")

    return organized_list

# Example usage
if __name__ == "__main__":
    latitude = 34.2
    longitude = -117.3
    start_year = 2020
    end_year = 2022 # Changed to 2022 to get more data

    print(f"Fetching data for Lat: {latitude}, Lon: {longitude}, Years: {start_year}-{end_year}")
    raw_api_data = fetch_nasa_power_data(latitude, longitude, start_year, end_year)

    if raw_api_data:
        # Uncomment the following line to see the full raw JSON response for detailed inspection
        # print("\nRaw data returned by API (for detailed inspection):")
        # print(json.dumps(raw_api_data, indent=2))

        organized_data = organize_nasa_power_data(raw_api_data)

        if organized_data:
            print("\nOrganized data:")
            for entry in organized_data:
                print(entry)
        elif organized_data == []:
            print("\nData organized, but no valid monthly entries were found.")
        else:
            print("\nOrganized data is None, indicating an issue with the organization function.")
    else:
        print("\nFailed to retrieve data from NASA POWER API. See detailed error message above.")

Fetching data for Lat: 34.2, Lon: -117.3, Years: 2020-2022

Organized data:
{'year': 2020, 'month': 1, 'ALLSKY_SFC_SW_DWN': 3.47, 'T2M': 10.79}
{'year': 2020, 'month': 2, 'ALLSKY_SFC_SW_DWN': 4.6, 'T2M': 12.2}
{'year': 2020, 'month': 3, 'ALLSKY_SFC_SW_DWN': 4.97, 'T2M': 11.32}
{'year': 2020, 'month': 4, 'ALLSKY_SFC_SW_DWN': 6.27, 'T2M': 15.27}
{'year': 2020, 'month': 5, 'ALLSKY_SFC_SW_DWN': 8.11, 'T2M': 19.45}
{'year': 2020, 'month': 6, 'ALLSKY_SFC_SW_DWN': 8.21, 'T2M': 21.2}
{'year': 2020, 'month': 7, 'ALLSKY_SFC_SW_DWN': 8.53, 'T2M': 24.57}
{'year': 2020, 'month': 8, 'ALLSKY_SFC_SW_DWN': 7.25, 'T2M': 27.34}
{'year': 2020, 'month': 9, 'ALLSKY_SFC_SW_DWN': 6.23, 'T2M': 26.71}
{'year': 2020, 'month': 10, 'ALLSKY_SFC_SW_DWN': 5.01, 'T2M': 21.96}
{'year': 2020, 'month': 11, 'ALLSKY_SFC_SW_DWN': 3.78, 'T2M': 14.8}
{'year': 2020, 'month': 12, 'ALLSKY_SFC_SW_DWN': 3.03, 'T2M': 11.4}
{'year': 2021, 'month': 1, 'ALLSKY_SFC_SW_DWN': 3.34, 'T2M': 10.75}
{'year': 2021, 'month': 2, 'ALLSKY_SFC_SW_